<a href="https://colab.research.google.com/github/saktiworkstation/road-to-ai-developer/blob/main/6_Sakti_22_11_4677pyspark_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sakti Kusuma Aji | 22.11.4677

In [28]:
!pip install pyspark

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import avg, when, count

In [30]:
spark = SparkSession.builder \
    .appName("Operasi Dasar Dataset PySpark") \
    .getOrCreate()

In [31]:
schema = StructType([
    StructField("ID", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Gender", StringType(), True)
])

In [32]:
df = spark.read.csv('/content/people.csv', header=True, schema=schema)

#Operasi Dasar

In [33]:
# Menampilkan skema DataFrame
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



In [34]:
# Menampilkan beberapa baris pertama dari dataset
df.show(10, truncate=False)

+---+---+----------------+------+
|ID |Age|Name            |Gender|
+---+---+----------------+------+
|0  |100|Penelope Lewis  |female|
|1  |101|David Anthony   |male  |
|2  |102|Ida Shipp       |female|
|3  |103|Joanna Moore    |female|
|4  |104|Lisandra Ortiz  |female|
|5  |105|David Simmons   |male  |
|6  |106|Edward Hudson   |male  |
|7  |107|Albert Jones    |male  |
|8  |108|Leonard Cavender|male  |
|9  |109|Everett Vadala  |male  |
+---+---+----------------+------+
only showing top 10 rows



In [35]:
# Menghitung jumlah record yang ada di DataFrame
jumlah_record = df.count()
print(f"Jumlah record: {jumlah_record}")

Jumlah record: 100000


In [36]:
# Melihat nama kolom
df_columns = df.columns
print(f"Kolom: {df_columns}")

Kolom: ['ID', 'Age', 'Name', 'Gender']


In [37]:
# Menampilkan ringkasan statistik dataset
df.describe().show()

+-------+-----------------+-----------------+-------------+------+
|summary|               ID|              Age|         Name|Gender|
+-------+-----------------+-----------------+-------------+------+
|  count|           100000|           100000|       100000| 98080|
|   mean|          49999.5|          50099.5|         NULL|  NULL|
| stddev|28867.65779668774|28867.65779668774|         NULL|  NULL|
|    min|                0|              100|Aaron Addesso|female|
|    max|            99999|           100099|  Zulma Biggs|  male|
+-------+-----------------+-----------------+-------------+------+



#Seleksi dan Filter Data

In [38]:
# Memilih kolom tertentu
df_selected = df.select("Name", "Age")
df_selected.show(10, truncate=False)

+----------------+---+
|Name            |Age|
+----------------+---+
|Penelope Lewis  |100|
|David Anthony   |101|
|Ida Shipp       |102|
|Joanna Moore    |103|
|Lisandra Ortiz  |104|
|David Simmons   |105|
|Edward Hudson   |106|
|Albert Jones    |107|
|Leonard Cavender|108|
|Everett Vadala  |109|
+----------------+---+
only showing top 10 rows



In [39]:
# Memfilter dataset untuk mendapatkan record di mana usia di atas 30
filtered_df = df.filter(df.Age > 30)
filtered_df.show(10, truncate=False)

+---+---+----------------+------+
|ID |Age|Name            |Gender|
+---+---+----------------+------+
|0  |100|Penelope Lewis  |female|
|1  |101|David Anthony   |male  |
|2  |102|Ida Shipp       |female|
|3  |103|Joanna Moore    |female|
|4  |104|Lisandra Ortiz  |female|
|5  |105|David Simmons   |male  |
|6  |106|Edward Hudson   |male  |
|7  |107|Albert Jones    |male  |
|8  |108|Leonard Cavender|male  |
|9  |109|Everett Vadala  |male  |
+---+---+----------------+------+
only showing top 10 rows



In [40]:
# Menghapus kolom "Gender"
df_dropped = df.drop("Gender")
df_dropped.show(10, truncate=False)

+---+---+----------------+
|ID |Age|Name            |
+---+---+----------------+
|0  |100|Penelope Lewis  |
|1  |101|David Anthony   |
|2  |102|Ida Shipp       |
|3  |103|Joanna Moore    |
|4  |104|Lisandra Ortiz  |
|5  |105|David Simmons   |
|6  |106|Edward Hudson   |
|7  |107|Albert Jones    |
|8  |108|Leonard Cavender|
|9  |109|Everett Vadala  |
+---+---+----------------+
only showing top 10 rows



#Operasi Pembersihan Data

In [41]:
# Menghapus nilai null dan menangani data yang hilang
cleaned_df = df.na.drop()

In [42]:
# Menggantikan nilai null pada kolom 'Age' dengan nilai rata-rata
average_age = df.select(avg("Age")).collect()[0][0]
df_filled = df.na.fill({"Age": int(average_age)})

#Transformasi Data

In [43]:
# Menambahkan kolom baru bernama "Category" berdasarkan "Age"
df_with_category = df_filled.withColumn("Category", when(df_filled.Age < 18, "Minor")
                                        .when((df_filled.Age >= 18) & (df_filled.Age <= 60), "Adult")
                                        .otherwise("Senior"))
df_with_category.show(10, truncate=False)

+---+---+----------------+------+--------+
|ID |Age|Name            |Gender|Category|
+---+---+----------------+------+--------+
|0  |100|Penelope Lewis  |female|Senior  |
|1  |101|David Anthony   |male  |Senior  |
|2  |102|Ida Shipp       |female|Senior  |
|3  |103|Joanna Moore    |female|Senior  |
|4  |104|Lisandra Ortiz  |female|Senior  |
|5  |105|David Simmons   |male  |Senior  |
|6  |106|Edward Hudson   |male  |Senior  |
|7  |107|Albert Jones    |male  |Senior  |
|8  |108|Leonard Cavender|male  |Senior  |
|9  |109|Everett Vadala  |male  |Senior  |
+---+---+----------------+------+--------+
only showing top 10 rows



In [47]:
# Mengganti nama kolom
renamed_df = df_with_category.withColumnRenamed("Age", "Umur")
renamed_df.show(10, truncate=False)

+---+----+----------------+------+--------+
|ID |Umur|Name            |Gender|Category|
+---+----+----------------+------+--------+
|0  |100 |Penelope Lewis  |female|Senior  |
|1  |101 |David Anthony   |male  |Senior  |
|2  |102 |Ida Shipp       |female|Senior  |
|3  |103 |Joanna Moore    |female|Senior  |
|4  |104 |Lisandra Ortiz  |female|Senior  |
|5  |105 |David Simmons   |male  |Senior  |
|6  |106 |Edward Hudson   |male  |Senior  |
|7  |107 |Albert Jones    |male  |Senior  |
|8  |108 |Leonard Cavender|male  |Senior  |
|9  |109 |Everett Vadala  |male  |Senior  |
+---+----+----------------+------+--------+
only showing top 10 rows



In [48]:
# Menghapus record yang duplikat
df_no_duplicates = renamed_df.dropDuplicates()
df_no_duplicates.show(10, truncate=False)

+----+----+--------------------+------+--------+
|ID  |Umur|Name                |Gender|Category|
+----+----+--------------------+------+--------+
|1426|1526|James Medina        |male  |Senior  |
|1443|1543|Janice Mcdavid      |female|Senior  |
|1728|1828|Ruth Breeden        |female|Senior  |
|2030|2130|Eleanor Luongo      |female|Senior  |
|2135|2235|Elizabeth Mckittrick|female|Senior  |
|2346|2446|Karen Colgan        |female|Senior  |
|2376|2476|Mary Williams       |female|Senior  |
|2432|2532|Eloisa Hedrick      |female|Senior  |
|2720|2820|James Mcgowen       |male  |Senior  |
|2935|3035|Suzanne Yang        |female|Senior  |
+----+----+--------------------+------+--------+
only showing top 10 rows



#Operasi Agregasi

In [49]:
# Menghitung jumlah orang per negara
grouped_df = df_with_category.groupBy("Category").count()
grouped_df.show(truncate=False)

+--------+------+
|Category|count |
+--------+------+
|Senior  |100000|
+--------+------+



In [50]:
# Melakukan agregasi menggunakan beberapa fungsi
df_agg = df_with_category.groupBy("Category").agg(
    avg("Age").alias("Rata_Rata_Usia"),
    count("Name").alias("Jumlah_Orang")
)
df_agg.show(truncate=False)

+--------+--------------+------------+
|Category|Rata_Rata_Usia|Jumlah_Orang|
+--------+--------------+------------+
|Senior  |50099.5       |100000      |
+--------+--------------+------------+



#Menggabungkan DataFrame

In [53]:
# Membuat DataFrame sampel untuk digabungkan
data_tambahan = [("Penelope Lewis", "Engineer"), ("Karen Colgan", "Doctor"), ("Suzanne Yang", "Teacher")]
skema_tambahan = StructType([
    StructField("Name", StringType(), True),
    StructField("Occupation", StringType(), True)
])
extra_df = spark.createDataFrame(data_tambahan, skema_tambahan)

In [54]:
# Melakukan join inner pada "Name"
joined_df = df_with_category.join(extra_df, on="Name", how="inner")
joined_df.show(truncate=False)

+--------------+----+----+------+--------+----------+
|Name          |ID  |Age |Gender|Category|Occupation|
+--------------+----+----+------+--------+----------+
|Penelope Lewis|0   |100 |female|Senior  |Engineer  |
|Karen Colgan  |2346|2446|female|Senior  |Doctor    |
|Suzanne Yang  |2935|3035|female|Senior  |Teacher   |
+--------------+----+----+------+--------+----------+



#Operasi SQL

In [55]:
# Mendaftarkan DataFrame sebagai Tampilan Sementara SQL
df_with_category.createOrReplaceTempView("people_view")

In [56]:
# Menjalankan query SQL pada tampilan yang terdaftar
sql_result = spark.sql("SELECT Name, Age FROM people_view WHERE Age > 30")
sql_result.show(truncate=False)

+-----------------+---+
|Name             |Age|
+-----------------+---+
|Penelope Lewis   |100|
|David Anthony    |101|
|Ida Shipp        |102|
|Joanna Moore     |103|
|Lisandra Ortiz   |104|
|David Simmons    |105|
|Edward Hudson    |106|
|Albert Jones     |107|
|Leonard Cavender |108|
|Everett Vadala   |109|
|Freddie Claridge |110|
|Annabelle Rosseau|111|
|Eulah Emanuel    |112|
|Shaun Love       |113|
|Alejandro Brennan|114|
|Robert Mcreynolds|115|
|Carla Spickard   |116|
|Florence Eberhart|117|
|Tina Gaskins     |118|
|Florence Mulhern |119|
+-----------------+---+
only showing top 20 rows



#Menyimpan DataFrame ke Berbagai Format

In [57]:
# Menyimpan DataFrame ke dalam format CSV
output_csv_path = "/content/cleaned_people_dataset.csv"
df_with_category.write.csv(output_csv_path, header=True)

In [58]:
# Menyimpan DataFrame ke dalam format Parquet
output_parquet_path = "/content/cleaned_people_dataset.parquet"
df_with_category.write.parquet(output_parquet_path)

In [59]:
# Menyimpan DataFrame ke dalam format JSON
output_json_path = "/content/cleaned_people_dataset.json"
df_with_category.write.json(output_json_path)